# HMDB database importer

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

In [ ]:
tree = ET.parse('hmdb_urine/urine_metabolites.xml')
root = tree.getroot()

In [552]:
def print_element(element, level=0):
    indent = "  " * level
    print(f"{indent}Tag: {element.tag}, Attributes: {element.attrib}")
    
    if element.text and element.text.strip():
        print(f"{indent}  Text: {element.text.strip()}")
    
    for child in element:
        print_element(child, level + 1)

first = True
for child in root:
    if first:
        print_element(child)
        first = False

Tag: {http://www.hmdb.ca}metabolite, Attributes: {}
  Tag: {http://www.hmdb.ca}version, Attributes: {}
    Text: 5.0
  Tag: {http://www.hmdb.ca}creation_date, Attributes: {}
    Text: 2005-11-16 15:48:42 UTC
  Tag: {http://www.hmdb.ca}update_date, Attributes: {}
    Text: 2021-10-13 17:34:04 UTC
  Tag: {http://www.hmdb.ca}accession, Attributes: {}
    Text: HMDB0000001
  Tag: {http://www.hmdb.ca}status, Attributes: {}
    Text: quantified
  Tag: {http://www.hmdb.ca}secondary_accessions, Attributes: {}
    Tag: {http://www.hmdb.ca}accession, Attributes: {}
      Text: HMDB00001
    Tag: {http://www.hmdb.ca}accession, Attributes: {}
      Text: HMDB0004935
    Tag: {http://www.hmdb.ca}accession, Attributes: {}
      Text: HMDB0006703
    Tag: {http://www.hmdb.ca}accession, Attributes: {}
      Text: HMDB0006704
    Tag: {http://www.hmdb.ca}accession, Attributes: {}
      Text: HMDB04935
    Tag: {http://www.hmdb.ca}accession, Attributes: {}
      Text: HMDB06703
    Tag: {http://www.hmdb

In [676]:
import pandas as pd
# List to hold metabolite data
# Function to extract text from an XML element, if it exists
def get_text(element, tag, namespace):
    child = element.find(f'ns:{tag}', namespace)
    return child.text if child is not None else None

# Function to extract a list of texts from XML elements, if they exist
def get_text_list(element, tag, namespace):
    children = element.findall(f'ns:{tag}', namespace)
    return [child.text for child in children] if children is not None else None

data = []
namespace = {'ns': 'http://www.hmdb.ca'}

# Iterate through each metabolite
for metabolite in root.findall('ns:metabolite', namespace):
    # Extract required elements
    accession = get_text(metabolite, 'accession', namespace)
    secondary_accessions = get_text_list(metabolite.find('ns:secondary_accessions', namespace), 'accession', namespace)
    name = get_text(metabolite, 'name', namespace)
    synonyms = get_text_list(metabolite.find('ns:synonyms', namespace), 'synonym', namespace)
    chemical_formula = get_text(metabolite, 'chemical_formula', namespace)
    iupac_name = get_text(metabolite, 'iupac_name', namespace)
    cas_registry_number = get_text(metabolite, 'cas_registry_number', namespace)
    smiles = get_text(metabolite, 'smiles', namespace)
    inchi = get_text(metabolite, 'inchi', namespace)
    inchikey = get_text(metabolite, 'inchikey', namespace)
    description = get_text(metabolite, 'description', namespace)
    monisotopic_molecular_weight = get_text(metabolite, 'monisotopic_molecular_weight', namespace)
    average_molecular_weight = get_text(metabolite, 'average_molecular_weight', namespace)
    
    # Append the metabolite data to the list
    data.append([accession, secondary_accessions, name, synonyms, chemical_formula, iupac_name, cas_registry_number, smiles, inchi, inchikey, monisotopic_molecular_weight, average_molecular_weight, description])

# Create a DataFrame
columns = ['accession', 'secondary_accessions', 'name', 'synonyms', 'chemical_formula', 'iupac_name', 'cas_registry_number', 'smiles', 'inchi', 'inchikey', 'monisotopic_molecular_weight', 'average_molecular_weight', 'description']
df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
df.head()

# Save the DataFrame to a CSV file if needed
#df.to_csv('metabolites.csv', index=False)
#df.drop('description').to_csv('metabolites_no_description.csv', index=False)

,accession,secondary_accessions,name,synonyms,chemical_formula,iupac_name,cas_registry_number,smiles,inchi,inchikey,monisotopic_molecular_weight,average_molecular_weight,description
0,HMDB0000001,"[HMDB00001, HMDB0004935, HMDB0006703, HMDB0006...",1-Methylhistidine,[(2S)-2-Amino-3-(1-methyl-1H-imidazol-4-yl)pro...,C7H11N3O2,(2S)-2-amino-3-(1-methyl-1H-imidazol-4-yl)prop...,332-80-9,CN1C=NC(C[C@H](N)C(O)=O)=C1,InChI=1S/C7H11N3O2/c1-10-3-5(9-4-10)2-6(8)7(11...,BRMWTNUJHUMWMS-LURJTMIESA-N,169.085126611,169.1811,"1-Methylhistidine, also known as 1-MHis or 1MH..."
1,HMDB0000002,"[HMDB00002, HMDB0060172, HMDB60172]","1,3-Diaminopropane","[1,3-Propanediamine, 1,3-Propylenediamine, Pro...",C3H10N2,"propane-1,3-diamine",109-76-2,NCCCN,InChI=1S/C3H10N2/c4-2-1-3-5/h1-5H2,XFNJVJPLKCPIBV-UHFFFAOYSA-N,74.08439833,74.1249,"1,3-Diaminopropane, also known as DAP or trime..."
2,HMDB0000005,"[HMDB00005, HMDB0006544, HMDB06544]",2-Ketobutyric acid,"[2-Ketobutanoic acid, 2-Oxobutyric acid, 3-Met...",C4H6O3,2-oxobutanoic acid,600-18-0,CCC(=O)C(O)=O,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,102.031694058,102.0886,"2-Ketobutyric acid, also known as alpha-ketobu..."
3,HMDB0000008,[HMDB00008],2-Hydroxybutyric acid,"[(S)-2-Hydroxybutanoic acid, 2-Hydroxybutyrate...",C4H8O3,(2S)-2-hydroxybutanoic acid,3347-90-8,CC[C@H](O)C(O)=O,"InChI=1S/C4H8O3/c1-2-3(5)4(6)7/h3,5H,2H2,1H3,(...",AFENDNXGAFYKQO-VKHMYHEASA-N,104.047344118,104.105,"2-Hydroxybutyric acid (CAS: 600-15-7), also kn..."
4,HMDB0000010,"[HMDB00010, HMDB0004990, HMDB0004991, HMDB0499...",2-Methoxyestrone,"[2-(8S,9S,13S,14S)-3-Hydroxy-2-methoxy-13-meth...",C19H24O3,"(1S,10R,11S,15S)-5-hydroxy-4-methoxy-15-methyl...",362-08-3,[H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[...,InChI=1S/C19H24O3/c1-19-8-7-12-13(15(19)5-6-18...,WHEUWNKSCXYKBU-QPWUGHHJSA-N,300.172544634,300.3921,2-Methoxyestrone (or 2-ME1) belongs to the cla...


In [ ]:
import os

def is_1h_file(path):
    if os.path.isfile(path):
        with open(path) as f:
            first_line = f.readline()
            return "_13C_" not in first_line and "ADDRESS" not in first_line
    return False

def list_files(folder_path):
    try:
        # Get all entries in the folder
        entries = os.listdir(folder_path)
        # Filter out only files
        files = [f for f in entries if is_1h_file(os.path.join(folder_path, f))]
        return files
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")
        return []
    except PermissionError:
        print(f"Permission denied: {folder_path}")
        return []

spectra_files = list_files('hmdb_urine/hmdb_nmr_peak_lists')
print(len(spectra_files))

1786


In [678]:
def create_dataframe(file_list):
    # List to hold the data for each file
    data = []

    for file_name in file_list:
        # Split the file name by "_" and take the first part as accession
        tokens  = file_name.split('_')
        # Append the data as a tuple (accession, file_name)
        data.append((tokens[0], file_name, tokens[1]))

    # Create the DataFrame
    df = pd.DataFrame(data, columns=['accession', 'file_name', 'dim'])

    return df

df_files = create_dataframe(spectra_files)

#Filter out only 1d files
df_files = df_files[df_files['dim'] == 'nmroned']
df_files.head()

,accession,file_name,dim
4,HMDB0003072,HMDB0003072_nmroned_5251_2734492.txt,nmroned
7,HMDB0000176,HMDB0000176_nmroned_1144_28428.txt,nmroned
9,HMDB0002873,HMDB0002873_nmroned_1910_33109.txt,nmroned
11,HMDB0000622,HMDB0000622_nmroned_1440_29992.txt,nmroned
12,HMDB0000548,HMDB0000548_nmroned_1415_29734.txt,nmroned


In [679]:
df = df.join(df_files.set_index('accession'), on='accession', how='left')
df = df.dropna(subset=['file_name'])
df.reset_index(inplace=True, drop=True)
df.head()

,accession,secondary_accessions,name,synonyms,chemical_formula,iupac_name,cas_registry_number,smiles,inchi,inchikey,monisotopic_molecular_weight,average_molecular_weight,description,file_name,dim
0,HMDB0000001,"[HMDB00001, HMDB0004935, HMDB0006703, HMDB0006...",1-Methylhistidine,[(2S)-2-Amino-3-(1-methyl-1H-imidazol-4-yl)pro...,C7H11N3O2,(2S)-2-amino-3-(1-methyl-1H-imidazol-4-yl)prop...,332-80-9,CN1C=NC(C[C@H](N)C(O)=O)=C1,InChI=1S/C7H11N3O2/c1-10-3-5(9-4-10)2-6(8)7(11...,BRMWTNUJHUMWMS-LURJTMIESA-N,169.085126611,169.1811,"1-Methylhistidine, also known as 1-MHis or 1MH...",HMDB0000001_nmroned_1022_27891.txt,nmroned
1,HMDB0000002,"[HMDB00002, HMDB0060172, HMDB60172]","1,3-Diaminopropane","[1,3-Propanediamine, 1,3-Propylenediamine, Pro...",C3H10N2,"propane-1,3-diamine",109-76-2,NCCCN,InChI=1S/C3H10N2/c4-2-1-3-5/h1-5H2,XFNJVJPLKCPIBV-UHFFFAOYSA-N,74.08439833,74.1249,"1,3-Diaminopropane, also known as DAP or trime...",HMDB0000002_nmroned_1023_27894.txt,nmroned
2,HMDB0000005,"[HMDB00005, HMDB0006544, HMDB06544]",2-Ketobutyric acid,"[2-Ketobutanoic acid, 2-Oxobutyric acid, 3-Met...",C4H6O3,2-oxobutanoic acid,600-18-0,CCC(=O)C(O)=O,"InChI=1S/C4H6O3/c1-2-3(5)4(6)7/h2H2,1H3,(H,6,7)",TYEYBOSBBBHJIV-UHFFFAOYSA-N,102.031694058,102.0886,"2-Ketobutyric acid, also known as alpha-ketobu...",HMDB0000005_nmroned_1024_27899.txt,nmroned
3,HMDB0000008,[HMDB00008],2-Hydroxybutyric acid,"[(S)-2-Hydroxybutanoic acid, 2-Hydroxybutyrate...",C4H8O3,(2S)-2-hydroxybutanoic acid,3347-90-8,CC[C@H](O)C(O)=O,"InChI=1S/C4H8O3/c1-2-3(5)4(6)7/h3,5H,2H2,1H3,(...",AFENDNXGAFYKQO-VKHMYHEASA-N,104.047344118,104.105,"2-Hydroxybutyric acid (CAS: 600-15-7), also kn...",HMDB0000008_nmroned_5245_2734397.txt,nmroned
4,HMDB0000010,"[HMDB00010, HMDB0004990, HMDB0004991, HMDB0499...",2-Methoxyestrone,"[2-(8S,9S,13S,14S)-3-Hydroxy-2-methoxy-13-meth...",C19H24O3,"(1S,10R,11S,15S)-5-hydroxy-4-methoxy-15-methyl...",362-08-3,[H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[...,InChI=1S/C19H24O3/c1-19-8-7-12-13(15(19)5-6-18...,WHEUWNKSCXYKBU-QPWUGHHJSA-N,300.172544634,300.3921,2-Methoxyestrone (or 2-ME1) belongs to the cla...,HMDB0000010_nmroned_1026_27907.txt,nmroned


In [680]:
df.shape

(608, 15)

In [ ]:
# Save the DataFrame to a CSV file if needed
df.to_csv('hmdb_urine/metabolites.csv', index=False)
df.drop(columns=['description']).to_csv('metabolites_no_description.csv', index=False)

## Prepare the dataset for approximate lookup using the metabolyte names and synomyms

In [16]:
df['synonyms_cat'] = df['name'] + " " + df['synonyms'].str.join(' ')
df['synonyms_cat'] = df['synonyms_cat'].str.lower()

In [13]:
from rapidfuzz import process, fuzz

# Function to perform approximate lookup
def approximate_lookup(df, column, search_string, scorer1, scorer2, limit=10):
    # Extract the column values as a list
    choices = df[column].str.lower().tolist()
    names = df['name'].values
    search_string = search_string.lower()
    # Get the best matches
    matches = process.extract(search_string, choices, scorer=scorer1, score_cutoff=100, limit=None)
    #print(matches)
    if len(matches) > 0:
        result = []
        for match in matches:
            matchings_names = df.loc[match[2],['synonyms']].values[0]
            best_name = ""
            best_score = 0
            for name in [names[match[2]]] + matchings_names:
                score = scorer2(search_string, name.lower())
                if score > best_score:
                    best_score = score
                    best_name = name
            result.append([names[match[2]], best_name, match[2], best_score,])
        # Map to the main name
        matches = pd.DataFrame(result).sort_values(3, ascending=False)
        #for synomym in data['synomyns'].
        return matches.head(limit).values
    return None

In [15]:
df['synonyms_cat']

0      1-methylhistidine [ ' ( 2 s ) - 2 - a m i n o ...
1      1,3-diaminopropane [ ' 1 , 3 - p r o p a n e d...
2      2-ketobutyric acid [ ' 2 - k e t o b u t a n o...
3      2-hydroxybutyric acid [ ' ( s ) - 2 - h y d r ...
4      2-methoxyestrone [ ' 2 - ( 8 s , 9 s , 1 3 s ,...
                             ...                        
603    2-methylhippuric acid [ ' n - ( o - t o l u o ...
604    glycyl-glycine [ ' 2 - ( a m i n o a c e t a m...
605    3-methoxybenzenepropanoic acid [ ' 3 - m e t h...
606    azithromycin [ ' ( 2 r , 3 s , 4 r , 5 r , 8 r...
607    cytarabine [ ' 1 - b e t a - d - a r a b i n o...
Name: synonyms_cat, Length: 608, dtype: object

In [14]:
matches = approximate_lookup(df, 'synonyms_cat', 'citric acid', fuzz.partial_token_ratio, fuzz.ratio, limit=3)
print(matches)

TypeError: can only concatenate list (not "str") to list

## Loook for p-cresol

In [725]:
#matches = approximate_lookup(df, 'name', "3-hydroxybutirate")
matches = approximate_lookup(df, 'synonyms_cat', "ethylendiaminotetraacetic acid", fuzz.partial_token_ratio, fuzz.ratio, limit=3)
print(matches)

[['Sarcosine' 'Methylaminoacetic acid' 140 80.76923076923077]
 ['Acetylglycine' 'Acetylaminoacetic acid' 208 76.92307692307692]
 ['Methylimidazoleacetic acid' 'Methylimidazoleacetic acid' 528 75.0]]


## Look for all metabolytes into the dataset

In [17]:
entries = ["2-Aminobutyric acid","2-Hydroxybutyric acid","2-Oxoglutaric acid","3-Hydroxybutyric acid","Acetic acid","Acetoacetic acid","Acetone","Alanine","Asparagine","Ca-EDTA","Choline","Citric acid","Creatine","Creatinine","D-Galactose","Dimethylsulfone","Ethanol","Formic acid","Glucose","Glutamic acid","Glutamine","Glycerol","Glycine","Histidine","Isoleucine","K-EDTA","Lactic acid","Leucine","Lysine","Methionine","N,N-Dimethylglycine","Ornithine","Phenylalanine","Proline","Pyruvic acid","Sarcosine","Succinic acid","Threonine","Trimethylamine-N-oxide","Tyrosine","Valine"]

for entry in entries:
    matches = approximate_lookup(df, 'synonyms_cat', entry, fuzz.partial_token_ratio, fuzz.ratio, limit=3)
    print(entry, matches)

TypeError: can only concatenate list (not "str") to list

In [ ]:
"L-Isoleucine", "L-Leucine", 'L-Valine', 'Isobutyric acid', '3-Hydroxybutyric acid', 'L-Lactic acid', 'L-Alanine', 'Acetic acid'

In [708]:
entries = ["Isoleucine","Leucine","Valine","Isoleucine","Valine","Isobutyrate","Ethanol","3-hydroxybutyrate","Lactate","Alanine","Acetate","3-hydroxybutyrate","Pyruvate","3-hydroxybutyrate","Succinate","Citrate","Citrate","Creatine","beta-Glucose","Ethanol","Alanine","Lactate","alpha-Glucose","Tyrosine","1-methylhistidine","Tyrosine","1-methylhistidine","Formate"]
for entry in entries:
    matches = approximate_lookup(df, 'synonyms_cat', entry, fuzz.partial_token_ratio, fuzz.ratio, limit=3)
    print(entry, matches)

Isoleucine [['L-Isoleucine' 'ISOLEUCINE' 88 100.0]
 ['L-Alloisoleucine' 'Isoleucine' 211 100.0]
 ['3-Methyl-2-oxovaleric acid' '2-Oxoisoleucine' 199 80.0]]
Leucine [['L-Leucine' 'LEUCINE' 253 100.0]
 ['L-Isoleucine' 'ISOLEUCINE' 88 82.35294117647058]
 ['L-Alloisoleucine' 'Isoleucine' 211 82.35294117647058]]
Valine [['L-Valine' 'VALINE' 336 100.0]
 ['alpha-Ketoisovaleric acid' 'Ketovaline' 9 75.0]
 ['L-Leucine' 'LEUCINE' 253 61.53846153846154]]
Isoleucine [['L-Isoleucine' 'ISOLEUCINE' 88 100.0]
 ['L-Alloisoleucine' 'Isoleucine' 211 100.0]
 ['3-Methyl-2-oxovaleric acid' '2-Oxoisoleucine' 199 80.0]]
Valine [['L-Valine' 'VALINE' 336 100.0]
 ['alpha-Ketoisovaleric acid' 'Ketovaline' 9 75.0]
 ['L-Leucine' 'LEUCINE' 253 61.53846153846154]]
Isobutyrate [['Isobutyric acid' 'Isobutyrate' 453 100.0]
 ['alpha-Hydroxyisobutyric acid' 'Hydroxyisobutyrate' 275
  75.86206896551724]
 ['2-Aminoisobutyric acid' 'a-Aminoisobutyrate' 470 75.86206896551724]]
Ethanol [['Ethanol' 'Ethanol' 50 100.0]
 ['Methan

In [ ]:
f = open('hmdb_urine/hmdb_nmr_peak_lists/HMDB0001370_nmroned_1690_31773.txt')# + df['file_name'].values[16], "r")
entry1 = f.read()
f.close()
print(entry1)

Table of Peaks 
No. 	(ppm) 	(Hz) 	Height 
1 	3.76 	2256.7 	0.1688 
2 	3.75 	2250.6 	0.3232 
3 	3.74 	2244.6 	0.1723 
4 	3.74 	2239.8 	0.1610 
5 	3.73 	2233.0 	0.2302 
6 	3.72 	2227.3 	0.1604 
7 	1.94 	1162.6 	0.0405 
8 	1.93 	1156.9 	0.0468 
9 	1.92 	1153.6 	0.0572 
10 	1.92 	1151.4 	0.0597 
11 	1.92 	1147.9 	0.1220 
12 	1.91 	1142.4 	0.1463 
13 	1.90 	1137.2 	0.1818 
14 	1.89 	1134.5 	0.1699 
15 	1.88 	1128.4 	0.1682 
16 	1.88 	1125.1 	0.0874 
17 	1.87 	1121.8 	0.1179 
18 	1.86 	1115.6 	0.0731 
19 	1.85 	1108.1 	0.0442 
20 	1.84 	1104.6 	0.0521 
21 	1.83 	1096.9 	0.0279 
22 	1.82 	1090.1 	0.0204 
23 	1.48 	884.7 	0.0301 
24 	1.46 	876.1 	0.0908 
25 	1.45 	868.4 	0.1181 
26 	1.43 	859.8 	0.0830 
27 	1.42 	851.9 	0.0456 

Table of Multiplets 
No. 	Shift1 (ppm) 	Hs 	Type 	J (Hz) 	Atom1 	Multiplet1 	(ppm) 
1 	1.45	2 	m 	- 	7 	M02 	1.36 .. 1.49 
2 	1.88	4 	m 	- 	8 6 	M01 	1.79 .. 1.96 
3 	3.74	2 	m 	- 	9 4 	M00 	3.70 .. 3.78 
 
Table of Assignments
No. 	Atom Exp. 	Shift (ppm) 	Multiplet 
1

# Guess the spectral frequency, compile the multiples, and recalculate the integrals that should match Hs, but sometimes has errors

In [432]:
import re
import pandas as pd
import math
import io

# Function to strip trailing commas
def strip_trailing_commas(s):
    if isinstance(s, str):
        return s.strip(',')
    return s


def parse_hmdb_data_clean(text, accession):
    """Parses HMDB NMR data and creates a data frame with peak information.

    Args:
        text: A string containing the HMDB NMR data in the specified format.

    Returns:
        A pandas DataFrame with one row per multiplet and columns for peak positions (ppm and Hz), heights, and assigned atoms.
    """
    text = text.lower()
    if( not( "peaks" in text )):
        text = "peaks" + text
        
    text = text.replace("muliplets", "multiplets")
    text = text.replace("mulitplets", "multiplets")
    text = text.replace("mnltiplets", "multiplets")
    text = text.replace("multuplets", "multiplets")
    text = text.replace("mutiplets", "multiplets")

    text = text.replace("praks", "peaks")
    text = text.replace("assignements", "assignments")
    
    text = text.replace("assignment\n", "assignments\n")

    text = text.replace("peaks", "peaks\n")
    text = text.replace("multiplets", "multiplets\n")
    text = text.replace("assignments", "assignments\n")
    text = text.replace("table", "\ntable")
    text = text.replace("atom exp.", "atom\texp.")

    #text = text.replace("\n\n\n", "\n\n")
    #text = text.replace(r"\n[\n]+", "\n\n")
    #print(text)
    try:
        # Extract tables using regular expressions
        table1 = re.search(r"peaks[\n ]+(.*?)\n([\t ]*)\n", text, re.DOTALL).group(1)
        #print(table1)
        table2 = re.search(r"multiplets[\n ]+(.*?)\n([\t ]*)\n", text, re.DOTALL).group(1)
        #print(multiplets_table)
        table3 = re.search(r"assignments[\n ]+(.*?)\n([\t ]*)\n", text, re.DOTALL)
        if table3 is not None:
           table3 = table3.group(1)
        else:
            table3 =  "no.\tatom\texp.shift(ppm)\tmultiplet\n1\t1\t1\1\ts0"
        #print(table2)
        # Guess which table is who
        assignments_table = None
        for table in [table1, table2, table3]:
            if "height" in table :
                peaks_table = table
            else:
                if "atom1" in table or "j (hz)" in table or "hs" in table:
                    multiplets_table = table
                else:
                    assignments_table = table
        if assignments_table == None:
            assignments_table = "no.\tatom\texp.shift(ppm)\tmultiplet\n1\t1\t1\1\ts0"
        
        multiplets_table = multiplets_table.replace("multiplet1 (ppm)", "multiplet1\t(ppm)")
        multiplets_table = re.sub(r"shift1[\s\t]*\(ppm\)", r"shift1(ppm)", multiplets_table)
        
        #print(assignments_table)

        #print(multiplets_table.replace(" ", ","))
        # Convert tables to pandas DataFrames
        peaks_df = pd.read_csv(io.StringIO(peaks_table.replace(" ", ",")), delim_whitespace=True)#sep="\t+", dtype=str)
        multiplets_df = pd.read_csv(io.StringIO(multiplets_table.replace(" ", ",")), delim_whitespace=True)
        assignments_df = pd.read_csv(io.StringIO(assignments_table.replace(" ", ",")), delim_whitespace=True)
        
        # Remove spaces in names
        peaks_df.columns = peaks_df.columns.str.replace(' ', '').str.replace(',', '')
        multiplets_df.columns = multiplets_df.columns.str.replace(' ', '').str.replace(',', '')
        assignments_df.columns = assignments_df.columns.str.replace(' ', '').str.replace(',', '')


        # Apply the function to each string column
        for col in peaks_df.select_dtypes(include=['object']).columns:
            peaks_df[col] = peaks_df[col].apply(strip_trailing_commas)
            
        # Apply the function to each string column
        for col in multiplets_df.select_dtypes(include=['object']).columns:
            multiplets_df[col] = multiplets_df[col].apply(strip_trailing_commas)  
        
        # Apply the function to each string column
        for col in assignments_df.select_dtypes(include=['object']).columns:
            assignments_df[col] = assignments_df[col].apply(strip_trailing_commas)  

        if "multiplet" not in assignments_df:
            assignments_df["multiplet"] = "x"

        peaks_df["(ppm)"] = pd.to_numeric(peaks_df["(ppm)"], errors='coerce')


        # Create an empty list to store the data for the new DataFrame
        data = []
    
        # Iterate over the multiplets
        for _, multiplet in multiplets_df.iterrows():
            if "m" not in multiplet["multiplet1"]:
                multiplet["(ppm)"] = multiplet["multiplet1"]
                multiplet["multiplet1"] = multiplet["atom1"]
                multiplet["atom1"] = ""
            if("atom1" not in multiplet):
                multiplet["atom1"] = ""
            if(isinstance(multiplet["atom1"], int) or isinstance(multiplet["atom1"], float)):
                multiplet["atom1"] = [multiplet["atom1"]]
            else:
                multiplet["atom1"] = multiplet["atom1"].split(",")

            #print(multiplet)
            if not ("j(hz)" in multiplet):
                multiplet["j(hz)"] = "-"
            if(isinstance(multiplet["j(hz)"], int) or isinstance(multiplet["j(hz)"], float)):
                 multiplet["j(hz)"] = [multiplet["j(hz)"]]
            else:
                multiplet["j(hz)"] = multiplet["j(hz)"].replace("-", "").split(",")
    
            ppm_range = multiplet["(ppm)"].replace(",", "").split("..")
            ppm_min = float(ppm_range[0])
            ppm_max = float(ppm_range[1])
            # Filter peaks within the multiplet range
            peaks_in_range = peaks_df[
                (peaks_df["(ppm)"] >= ppm_min) & (peaks_df["(ppm)"] <= ppm_max)
            ]
    
            # Extract peak information
            ppm_values = peaks_in_range["(ppm)"].tolist()
            heights = peaks_in_range["height"].tolist()
            if ("(hz)" in peaks_in_range):
                hz_values = peaks_in_range["(hz)"].tolist()
            else:
                hz_values = np.array(ppm_values) * 601
    
            # Find assigned atoms
            assigned_atoms = assignments_df[
                assignments_df["multiplet"] == multiplet["multiplet1"]
            ]["atom"].tolist()

            if "shift1(ppm)" not in multiplet:
                multiplet["shift1(ppm)"] = (ppm_min + ppm_max) / 2
            # Append data for the current multiplet to the list
            data.append(
                {
                    "accession": accession,
                    "multiplet": multiplet["multiplet1"],
                    "shift1(ppm)": multiplet["shift1(ppm)"],
                    "hs": multiplet["hs"],
                    "j(hz)": multiplet["j(hz)"],
                    "atom1": multiplet["atom1"],
                    "type": multiplet["type"],
                    "from": ppm_min,
                    "to": ppm_max,
                    "ppm": ppm_values,
                    "hz": hz_values,
                    "heights": heights,
                    "assigned atoms": assigned_atoms,
                }
            )
    
        # Create the final DataFrame
        return pd.DataFrame(data)
    except Exception as e:
        #print(text)
        print(e)
        return None;

In [433]:
assignment_entry = parse_hmdb_data_clean(
"""
Table of Peaks 
No. 	(ppm) 	(Hz) 	Height 
1 	3.76 	2256.7 	0.1688 
2 	3.75 	2250.6 	0.3232 
3 	3.74 	2244.6 	0.1723 
4 	3.74 	2239.8 	0.1610 
5 	3.73 	2233.0 	0.2302 
6 	3.72 	2227.3 	0.1604 
7 	1.94 	1162.6 	0.0405 
8 	1.93 	1156.9 	0.0468 
9 	1.92 	1153.6 	0.0572 
10 	1.92 	1151.4 	0.0597 
11 	1.92 	1147.9 	0.1220 
12 	1.91 	1142.4 	0.1463 
13 	1.90 	1137.2 	0.1818 
14 	1.89 	1134.5 	0.1699 
15 	1.88 	1128.4 	0.1682 
16 	1.88 	1125.1 	0.0874 
17 	1.87 	1121.8 	0.1179 
18 	1.86 	1115.6 	0.0731 
19 	1.85 	1108.1 	0.0442 
20 	1.84 	1104.6 	0.0521 
21 	1.83 	1096.9 	0.0279 
22 	1.82 	1090.1 	0.0204 
23 	1.48 	884.7 	0.0301 
24 	1.46 	876.1 	0.0908 
25 	1.45 	868.4 	0.1181 
26 	1.43 	859.8 	0.0830 
27 	1.42 	851.9 	0.0456 

Table of Multiplets 
No. 	Shift1 (ppm) 	Hs 	Type 	J (Hz) 	Atom1 	Multiplet1 	(ppm) 
1 	1.45 2 	m 	- 	7 	M02 	1.36 .. 1.49 
2 	1.88 4 	m 	- 	8 6 	M01 	1.79 .. 1.96 
3 	3.74 2 	m 	- 	9 4 	M00 	3.70 .. 3.78 
 
Table of Assignments
No. 	Atom Exp. 	Shift (ppm) 	Multiplet 
1 	4 	3.74 	M00 
2 	6 	1.88 	M01 
3 	7 	1.45 	M02 
4 	8 	1.88 	M01 
5 	9 	3.74 	M00 

""", "HMDB1")
assignment_entry

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
0,HMDB1,m02,"1.45,2",m,[7],[],-,1.36,1.49,"[1.48, 1.46, 1.45, 1.43, 1.42]","[884.7, 876.1, 868.4, 859.8, 851.9]","[0.0301, 0.0908, 0.1181, 0.0830, 0.0456]",[]
1,HMDB1,m01,"1.88,4",m,"[8, 6]",[],-,1.79,1.96,"[1.94, 1.93, 1.92, 1.92, 1.92, 1.91, 1.9, 1.89...","[1162.6, 1156.9, 1153.6, 1151.4, 1147.9, 1142....","[0.0405, 0.0468, 0.0572, 0.0597, 0.1220, 0.146...",[]
2,HMDB1,m00,"3.74,2",m,"[9, 4]",[],-,3.70,3.78,"[3.76, 3.75, 3.74, 3.74, 3.73, 3.72]","[2256.7, 2250.6, 2244.6, 2239.8, 2233.0, 2227.3]","[0.1688, 0.3232, 0.1723, 0.1610, 0.2302, 0.1604]",[]


In [430]:
assignment_entry = parse_hmdb_data_clean(entry1 + "\n\n", "HMDB1")
assignment_entry.head()

no.,	shift1(ppm),	hs,	type,	j,(hz),	atom1,	multiplet1,	(ppm),
1,	1.45	2,	m,	-,	7,	m02,	1.36,..,1.49,
2,	1.88	4,	m,	-,	8,6,	m01,	1.79,..,1.96,
3,	3.74	2,	m,	-,	9,4,	m00,	3.70,..,3.78,


,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
0,HMDB1,m02,1.45,2,[],[7],m,1.36,1.49,"[1.48, 1.46, 1.45, 1.43, 1.42]","[884.7, 876.1, 868.4, 859.8, 851.9]","[0.0301, 0.0908, 0.1181, 0.0830, 0.0456]",[]
1,HMDB1,m01,1.88,4,[],"[8, 6]",m,1.79,1.96,"[1.94, 1.93, 1.92, 1.92, 1.92, 1.91, 1.9, 1.89...","[1162.6, 1156.9, 1153.6, 1151.4, 1147.9, 1142....","[0.0405, 0.0468, 0.0572, 0.0597, 0.1220, 0.146...",[]
2,HMDB1,m00,3.74,2,[],"[9, 4]",m,3.70,3.78,"[3.76, 3.75, 3.74, 3.74, 3.73, 3.72]","[2256.7, 2250.6, 2244.6, 2239.8, 2233.0, 2227.3]","[0.1688, 0.3232, 0.1723, 0.1610, 0.2302, 0.1604]",[]


# Index the whole db

In [ ]:
nmrdb = None
HMDB_PATH = 'hmdb_urine/hmdb_nmr_peak_lists/'
index = 0
count_errors = 0

for entry in df.iterrows():
    index += 1
    #531, 582
    if(index in [69, 301, 365, 387, 409, 412, 465, 493, 500, 501, 515, 531, 563, 580, 581]):
        continue
    f = open(HMDB_PATH + entry[1]['file_name'], "r")
    content = f.read()
    f.close()
    assignment_entry = parse_hmdb_data_clean(content + "\n\n", entry[1]['accession'])
    if assignment_entry is not None:
        if nmrdb is None:
            nmrdb = assignment_entry
        else:
            nmrdb = pd.concat([nmrdb, assignment_entry])
    else:
        count_errors += 1
        print(index)
        print(HMDB_PATH + entry[1]['file_name'])
        print(content)
        break;

nmrdb['from'] = pd.to_numeric(nmrdb['from'], errors='coerce')
nmrdb['to'] = pd.to_numeric(nmrdb['to'], errors='coerce')
nmrdb['shift1(ppm)'] = pd.to_numeric(nmrdb['shift1(ppm)'], errors='coerce')

nmrdb = nmrdb.sort_values("shift1(ppm)").reset_index(drop=True)

nmrdb.to_csv('inst/spectral1hnmr.csv', index=False)
print(count_errors, index)
nmrdb.shape

0 608


(3208, 13)

In [4]:
import bisect

# Range query function
def range_query_and(ranges, nmrdb):
    indexer = [(a, b) for a, b in zip(nmrdb['shift1(ppm)'].values, nmrdb.index.values)]
    result_ids = None
    for rng in ranges:
        left_index = bisect.bisect_left(indexer,  (rng[0], -1))
        right_index = bisect.bisect_right(indexer, (rng[1], float('inf')))
        res = set(nmrdb.loc[[item[1] for item in indexer[left_index:(right_index+1)]],['accession']]['accession'].values)
        if result_ids is None:
            result_ids = res
        else:
            result_ids = result_ids & res
    return list(result_ids)

def multiple_query(query, nmrdb, metabolytes):
    res = range_query_and(query, nmrdb)
    filtered_df = metabolytes.loc[metabolytes['accession'].isin(res), ]
    result = []
    for row in filtered_df.iterrows():
        signals = nmrdb[nmrdb['accession'] == row[1]["accession"]]
        matches = 0
        for query_i in query:
            from_i = query_i[0]
            to_i = query_i[1]
            mul_i = query_i[2]
            for signal in signals.iterrows():
                if signal[1]['shift1(ppm)'] >= from_i and signal[1]['shift1(ppm)'] <= to_i and signal[1]['type'] == mul_i:
                    matches += 1
                    break
        result.append((row[1]["accession"], row[1]["name"], len(query) / signals.shape[0] + matches /  len(query)))
                    
    result = pd.DataFrame(result, columns=["accession", "name", "similarity"])
    result = result.sort_values("similarity", ascending=False)
    return result


In [8]:
import pandas as pd

nmrdb = pd.read_csv("inst/spectral1hnmr.csv")
df = pd.read_csv("inst/metabolites.csv")

In [22]:
res = df[df['name'].isin(["L-Isoleucine", "L-Leucine", 'L-Valine', 'Isobutyric acid', '3-Hydroxybutyric acid', 'L-Lactic acid', 'L-Alanine', 'Acetic acid'])]
res = res['accession']
res

5      HMDB0000011
20     HMDB0000042
81     HMDB0000161
88     HMDB0000172
96     HMDB0000190
253    HMDB0000687
336    HMDB0000883
453    HMDB0001873
Name: accession, dtype: object

In [33]:
nmrlist = nmrdb[nmrdb['accession'].isin(res.values)]
nmrlist = nmrlist[['accession', 'ppm', 'heights', 'hs']]

nmrlist


,accession,ppm,heights,hs
131,HMDB0000172,"[0.913, 0.926, 0.938]","['1.7642', '3.9743', '2.0355']",4
148,HMDB0000687,"[0.936, 0.948, 0.96]","['1.9662', '4.0000', '2.1277']",57
163,HMDB0000883,"[0.969, 0.983]","[0.9681, 1.0]",3
176,HMDB0000172,"[0.991, 1.003]","['3.8784', '4.0000']",4
190,HMDB0000883,"[1.022, 1.036]","[0.953, 0.9614]",3
265,HMDB0000011,"[1.199, 1.209]","[0.9939, 1.0]",3
272,HMDB0001873,"[1.2, 1.21]","['0.9989', '0.9629']",5
285,HMDB0000172,"[1.21, 1.223, 1.235, 1.238, 1.245, 1.25, 1.257...","['0.0754', '0.2227', '0.2706', '0.2757', '0.31...",1
344,HMDB0000190,"[1.31, 1.324]","['0.9981', '1.0000']",4
412,HMDB0000172,"[1.423, 1.432, 1.436, 1.444, 1.448, 1.457, 1.4...","['0.0865', '0.1094', '0.2685', '0.3115', '0.30...",1


## Rank matches according to multiplet similarity

In [9]:
query = [(2.5, 2.6, 'd'), (2.6, 2.7, 'd')]

result = multiple_query(query, nmrdb, df)
result.head()        

NameError: name 'multiple_query' is not defined

In [9]:
query = [(1.25, 1.35, 'd'), (4.05, 4.15, 'q')]

result = multiple_query(query, nmrdb, df)
result.head(6)    

,accession,name,similarity
1,HMDB0000190,L-Lactic acid,2.000000
0,HMDB0000030,Biotin,0.700000
2,HMDB0000701,Hexanoylglycine,0.400000
4,HMDB0002802,Cortisone,0.142857
3,HMDB0001348,SM(d18:1/18:0),0.133333
5,HMDB0014352,Azithromycin,0.062500


In [ ]:
import pandas as pd
import re

def count_atoms(formula):
    """
    Counts the number of atoms of each element in a chemical formula.

    Args:
        formula: A string representing the chemical formula.

    Returns:
        A dictionary where keys are element symbols and values are the number of atoms of that element.
    """
    atoms = {}
    for match in re.findall(r"([A-Z][a-z]*)(\d*)", formula):
        element = match[0]
        count = int(match[1]) if match[1] else 1
        atoms[element] = atoms.get(element, 0) + count
    return atoms

# Apply the function to the 'chemical_formula' column
atom_counts = df['chemical_formula'].apply(count_atoms)

# Expand the dictionary of atom counts into separate columns
#df = pd.concat([df, atom_counts.apply(pd.Series)], axis=1)
#print(df)